In [1]:
import sys
from gene import Gene, GeneChain
from evolution import Evolution
from loader import load_results, load_trading_params, load_nontrading_params, load_indivs
from crosser import Cross10to100, RandomGeneCrosser
from mutator import GausianMutator, MutRandomIndivs, RandomGeneChainMutator
from selector import TopNSelector
from util import print_row, print_header
from fitness import evaluate, Evaluation
from copy import deepcopy
import numpy as np
import datetime

## Load data

In [2]:
db_indivs = load_indivs()   # pandas object, for fitness function; db individuals have only non-trading parameters
trading_params = load_trading_params()  # pandas object, for precomputation
nontrading_params = load_nontrading_params()    # pandas object, for precomputation

In [3]:
db_indivs.head(3)

,0,1,2,3,4,5,6,7,8
0,3,21,5,0,1.34,0.09,0.38,0.38,0.16
1,4,21,5,0,1.42,0.10,0.40,0.40,0.33
2,5,21,5,0,1.42,0.21,0.40,0.40,0.33


In [4]:
trading_params

,TYPE,MIN,MAX,DESCRIPTION
0,float64,-1.0,1.0,distance between order price and level
1,float64,0.5,2.0,stop loss
2,float64,0.5,2.5,stop out
3,float64,0.5,3.0,take profit
4,float64,-2.0,2.0,bu level
5,float64,0.5,3.0,bu condition level
6,float64,0.5,3.0,minimal range between orders with the same di...
7,int64,1.0,5.0,max stops
8,int64,300.0,3600.0,expiration(in minuts)
9,int64,0.0,1500.0,min time between orders(in minuts)


In [5]:
nontrading_params

,TYPE,MIN_RANGE,DESCRIPTION
0,int64,4.0,bars on level after fracture
1,int64,10.0,bars before level behind
2,int64,2.0,bars between fracture and touch
3,int64,1.0,number of lps bigger than 0.5
4,float64,0.4,fracture rollback
5,float64,0.4,consolidation rollback
6,float64,0.4,max lp
7,float64,0.4,max lp in 3 first bars
8,float64,0.4,max close behind level


## Init base gene chain

In [6]:
base_gene_chain = GeneChain()
for index, row in nontrading_params.iterrows():
    dtype = np.dtype(row["TYPE"]).type
    is_interval = True if dtype != bool else False
    min_range = dtype(row["MIN_RANGE"])
    max_value = dtype(max(db_indivs[index]))
    min_value = dtype(min(db_indivs[index]))

    base_gene_chain.add(Gene(dtype=dtype, is_interval=is_interval, min_radius=min_range/2,
                             min_value=min_value, max_value=max_value, ordering=-1))

for index, row in trading_params.iterrows():
    is_interval = False
    dtype = np.dtype(row["TYPE"]).type
    min_value = dtype(row["MIN"])
    max_value = dtype(row["MAX"])
    base_gene_chain.add(Gene(dtype=dtype, is_interval=is_interval, tags=["trading"],
                             min_value=min_value, max_value=max_value, ordering=-1))

#for gene in base_gene_chain:
#    print(gene)

## Init genetic algorithm

In [7]:
ev = Evolution(
    MAX_GENS=100,
    evaluation=Evaluation(spread=0.0001),
    base_indiv=base_gene_chain,
    crosser=RandomGeneCrosser(),
    selector=TopNSelector(),
    verbose=True)

In [8]:
#ev.init_random_indivs(size=100, min_models=50)
#ev.dump_indivs("population.pkl")
ev.load_indivs("population.pkl", 10)

## Run 

In [9]:
ev.run()

Generation: 100 | Best fitness: 317.0 | Models: 744 | Time: avg per gen = 142s. total = 12942s.)                                                                                                    

In [11]:
ev.dump_indivs("best.pkl")

In [13]:
ev.print_generation()

fitness: 317.0 models: 744 max account value: 317.5 max drawdown: 9.7
fitness: 312.4 models: 748 max account value: 318.2 max drawdown: 7.7
fitness: 312.4 models: 748 max account value: 318.2 max drawdown: 7.7
fitness: 312.4 models: 748 max account value: 318.2 max drawdown: 7.7
fitness: 312.4 models: 748 max account value: 318.2 max drawdown: 7.7
fitness: 312.4 models: 748 max account value: 318.2 max drawdown: 7.7
fitness: 312.4 models: 748 max account value: 318.2 max drawdown: 7.7
fitness: 312.4 models: 748 max account value: 318.2 max drawdown: 7.7
fitness: 312.4 models: 748 max account value: 318.2 max drawdown: 7.7
fitness: 312.4 models: 748 max account value: 318.2 max drawdown: 7.7
